In [1]:
import pandas as pd
import numpy as np
import time
import requests
import urllib.request
from urllib.parse import quote_plus
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import unicodedata

In [2]:
tag_table = pd.read_excel('해시태그 정리.xlsx')
tag_table

,일상,육아,여행/장소,취미/예술,패션,음식,애완동물
0,일상,육아,대구,럽스타그램,데일리룩,먹스타그램,멍스타그램
1,데일리,육아스타그램,홍대,사진,오오티디,맛스타그램,고양이
2,셀카,육아소통,강남,사랑해,패션,카페,반려견
3,셀스타그램,딸스타그램,광주,운동,옷스타그램,먹방,강아지
4,셀피,육아맘,해운대,네일아트,NaN,맛집,냥스타그램
5,얼스타그램,맘스타그램,제주도,젤네일,NaN,커피,개스타그램
6,일상스타그램,아들스타그램,서면,인테리어,NaN,카페스타그램,펫스타그램
7,주말,도치맘,대전,운동하는여자,NaN,NaN,댕댕이
8,다이어트,애스타그램,이태원,네일,NaN,NaN,NaN
9,데이트,인스타베이비,제주도,방탄소년단,NaN,NaN,NaN


In [4]:
### 설정값
#이미지를 따로 저장할 'img'라는 디렉토리를 미리 생성해주세요.

path = '/chromedriver.exe' #크롬드라이버 위치

ID = '' #인스타 id
PW = '' #인스타 비밀번호

SCROLL_NUM = 1 #얼마나 스크롤할 지 정하기(스크롤을 내려 게시물을 많이 확보해야 합니다. 값이 최소 1이어야 합니다.)
CRAWLING_NUM = 3 #크롤링할 게시물 개수

tag_list = tag_table['육아'][1:3].dropna() #검색할 태그 column명과 인덱싱 해주세요.
tag_list2 = tag_table['취미/예술'][:3].dropna()
tag_list = pd.concat([tag_list,tag_list2],ignore_index=True)
tag_list

0    육아스타그램
1      육아소통
2     럽스타그램
3        사진
4       사랑해
dtype: object

In [5]:
def get_url(driver, tag):
    url = "https://www.instagram.com/explore/tags/"
    url = url + quote_plus(tag)
    driver.get(url)
    time.sleep(3)

    html = driver.page_source
    soup = bs(html, 'lxml')

In [6]:
def scroll(drive, scroll_num):
    
    #SCROLL_NUM만큼 맨 밑으로 스크롤하고 맨 위로 돌아오기
    for _ in range(scroll_num):
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(1)
        
    driver.execute_script('window.scrollTo(0,0);')

    #첫 번째 게시물
    driver.find_element_by_class_name('_9AhH0').click()

In [7]:
def crawl(driver, crawl_num, tag):
    
    n = 0
    
    for i in range(crawl_num):
        html = driver.page_source
        soup = bs(html, 'lxml')
        time.sleep(1)

        img_url = soup.select('.KL4Bh')[n].img['src']
        image_name = tag + '_' + str(n) + '.jpg'
        with urllib.request.urlopen(img_url) as f:
            with open('./img/' + tag + '_' + str(n) + '.jpg', 'wb') as h:
                img = f.read()
                h.write(img)

        location = driver.find_elements_by_class_name('O4GlU')
        if len(location) == 0:
            location = np.nan
        else:
            location = location[0].text
            location = unicodedata.normalize('NFC', location)

        contents = driver.find_elements_by_xpath('/html/body/div[5]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span')
        if len(contents) == 0:
            contents = np.nan
        else:
            contents = contents[0].text
            contents = unicodedata.normalize('NFC', contents)

        #답글 열기
        replies = driver.find_elements_by_class_name('EizgU')
        for reply in replies:
            reply.click()
        time.sleep(1)

        hashtags = ''
        tag_list = driver.find_elements_by_class_name('xil3i')
        if len(tag_list) == 0:
            hashtags = np.nan
        else:    
            for k in range(len(tag_list)):
                hashtags += tag_list[k].text
            hashtags = unicodedata.normalize('NFC', hashtags)

        image_url_list.append(img_url)
        image_name_list.append(image_name)
        location_list.append(location)
        hashtags_list.append(hashtags)
        contents_list.append(contents)

        #다음 게시물
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(1)
        
        n += 1

In [8]:
### 실행코드
image_name_list = []
image_url_list = []
contents_list = []
location_list = []
hashtags_list = []

driver = webdriver.Chrome(path)
driver.maximize_window()
driver.get('https://instagram.com/explore')
time.sleep(1)

driver.find_element_by_name('username').send_keys(ID)
driver.find_element_by_name('password').send_keys(PW)
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button').click()
time.sleep(3)

for tag in tag_list:
    get_url(driver, tag)
    time.sleep(1)
    
    scroll(driver, SCROLL_NUM)
    crawl(driver, CRAWLING_NUM, tag)

In [19]:
### 저장
data = {'image_name':image_name_list,
        'image_url':image_url_list,
         'content':contents_list,
         'location':location_list,
         'hashtags':hashtags_list}
    
df = pd.DataFrame(data)
df = df.drop_duplicates(['image_url'])
df.to_csv("data.csv", index=True, encoding='utf-8-sig')

In [20]:
df

,image_name,image_url,content,location,hashtags
0,육아스타그램_0.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,400일차 어피치 아쥰💜\n\n#생후400일,NaN,#생후400일#육아#육아일상#육아소통#육아템#육아꿀템#육아스타그램#아기스타그램#아들...
1,육아스타그램_1.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,💕\n\nㆍ\nㆍ내 초등학교 친구😎(나보다 작았던ㅎ)\nㆍ내 중딩 내가 괴롭?히던 ...,NaN,#젊줌마#daily#소통#일상스타그램#데일리#먹팔#육아스타그램#아들스타그램#seli...
2,육아스타그램_2.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,단아네💕\n순둥순동 다시 뽀해졌네\n오늘은 머하고 놀까 단아야😍,지웰시티(Gwell City),#단아네#소통#육아#맞팔#육아소통#육아맞팔#협찬#딸맘#도치맘#설날#육아스타그램#맘스...
4,육아소통_1.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,-\n⠀\n𝕋𝕠𝕣𝕚 D+11💘\n⠀\n⠀\n엄마는 제 안티인가봐요ㅋㅋㅋㅋㅋ\n사진을...,NaN,#임신#출산#출산다이어트#조리원#조리원생활#조리원라이프#조리원천국#모유라떼🍼#아기스...
5,육아소통_2.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,오늘도 너무 기분좋은 우리 휘✨\n휘는 아침에 일어나면 20분도 넘게 침대에 혼자 ...,Yeongtong-gu,#4개월아기#태어난지140일째#아침#휘✨#20년생#20년생아기#쥐띠아기#쥐띠맘#육아...
6,럽스타그램_0.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,세윤이가 맘에든댕❤️,익선동,#일상스타그램#일상#daily#셀카그램#셀카#selca#selfie#팔로우#foll...
7,럽스타그램_1.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,꿀떨어지네 꿀꿀🍯,흰여울문화마을,#럽스타그램#감성#분위기#instadaily#데일리#일상#맞팔#선팔#선팔하면맞팔#좋...
8,럽스타그램_2.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,❤️\n(사진만 찍고 마스크 바로 썼어요😷),행궁동,#행궁동#럽스타그램#커플#01년생#00년생#02년생#ootd#좋아요테러#좋아요반사#...
9,사진_0.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,할일: 누워있기,NaN,#daily#selfie#일상#셀카#코디#사진#셀스타그램#ootd#likeforli...
10,사진_1.jpg,https://scontent-ssn1-1.cdninstagram.com/v/t51...,술김에.\n\n.\n.\n.\n.\n.\n\n#구름 #구름사진 #하늘 #하늘사진 #...,NaN,#구름#구름사진#하늘#하늘사진#사진#사진스타그램#사진소통#사진작가#사진계정맞팔#사진...
